# Master dataframe en functie om forecast 2021 2023 van verbruik te maken



In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import time

# show all columns in the dataframe
pd.set_option('max_columns', None)

In [2]:
# extra imports 

#!pip install altair
import altair as alt


import warnings
warnings.filterwarnings("ignore")

# Define Predict Function 

In [261]:
# Define function that returns a low, mid or high prediction of SJV_TOTAAL for each PC6 for the years 2021-2023 on the basis
# of the electricity consumption in period 2010-2020. Default prediction type is 'mid'. 

import pystan
import fbprophet
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


def predict_verbruik_prophet(df_input, predict_type='mid'):

    # PROPHET PREDICTION MODEL
    
    # Copy input dataframe
    df = df_input.copy()
    
    # Create a unique pc6 identifier
    sep="-"
    df['match'] = df['POSTCODE_VAN']    
    df['match'] = df['match'].astype(str) + sep + df['POSTCODE_TOT'].astype(str)
    list_of_pc6= df['match'].unique()
    
    print (f"\nForecasting {predict_type} case for 2021-2023 for {len(list_of_pc6)} pc6's")
   
    # Create dataframe for forecast 2021-2023
    df_forecast = pd.DataFrame()
    df_forecast.loc[:,'ds'] = [2021, 2022, 2023]
    
    #Create output dataframe
    df_output = pd.DataFrame()

    
    # Loop over each pc6 and predict 2021-2023      
    for pc6 in list_of_pc6:
   
        df_predict = pd.DataFrame()
        df_predict['ds'] = df['JAAR'][df['match'] == pc6].copy()
        df_predict['y']  = df['SJV_TOTAAL'][df['match'] == pc6]
        
        # We fit the model by instantiating a new Prophet object.
        # Any settings to the forecasting procedure are passed into the constructor. 
        # Then you call its fit method and pass in the historical dataframe. Fitting should take 1-5 seconds.
        
        print (f"Forecasting {predict_type} case for 2021-2023 with prophet model for pc6 {pc6}")
        m = Prophet(growth='linear',
                    changepoints=None,
                    n_changepoints=2, #25,
                    changepoint_range=0.4, #0.8,
                    yearly_seasonality=True, #'auto',
                    weekly_seasonality=False, #'auto',
                    daily_seasonality=False, #'auto',
                    holidays=None , #holidays,
                    seasonality_mode='additive',
                    seasonality_prior_scale=10.0,
                    holidays_prior_scale=10.0,
                    changepoint_prior_scale=0.05,
                    mcmc_samples=0,
                    interval_width=0.8,
                    uncertainty_samples=1000,
                    stan_backend=None)
        m.fit(df_predict)
        
        # Create rows of output dataframe for 2021-2023
        df_copy = pd.DataFrame(df[(df['match'] == pc6) & (df['JAAR'] == 2020)])       
        df_temp = pd.DataFrame(np.repeat(df_copy.values,3,axis=0))
        df_temp.columns = df_copy.columns
        df_temp['JAAR'].iloc[0] = 2021
        df_temp['JAAR'].iloc[1] = 2022
        df_temp['JAAR'].iloc[2] = 2023
        
        # Calculate forecast 
        forecast = m.predict(df_forecast) 
        
        if predict_type == "low":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast['yhat_lower'].iloc[0]
           df_temp['SJV_TOTAAL'].iloc[1] = forecast['yhat_lower'].iloc[1]
           df_temp['SJV_TOTAAL'].iloc[2] = forecast['yhat_lower'].iloc[2]
        elif predict_type == "mid":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast['yhat'].iloc[0]
           df_temp['SJV_TOTAAL'].iloc[1] = forecast['yhat'].iloc[1]
           df_temp['SJV_TOTAAL'].iloc[2] = forecast['yhat'].iloc[2]
        elif predict_type == "high":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast['yhat_upper'].iloc[0]
           df_temp['SJV_TOTAAL'].iloc[1] = forecast['yhat_upper'].iloc[1]
           df_temp['SJV_TOTAAL'].iloc[2] = forecast['yhat_upper'].iloc[2]
        else:
           print(f"Unexpected prediction type {predict_type}")
       
        #print(df_temp)
    
        df_output = df_output.append(df_temp)  
        
    # Delete match column
    del df_output['match']
    
    return df_output


def predict_verbruik(df_input, predict_type='mid'):

    # LINEAR REGRESSION MODEL

    
    # Copy input dataframe
    df = df_input.copy()
    
    # Create a unique pc6 identifier
    sep="-"
    df['match'] = df['POSTCODE_VAN']    
    df['match'] = df['match'].astype(str) + sep + df['POSTCODE_TOT'].astype(str)
    list_of_pc6= df['match'].unique()
    
    print (f"\nForecasting {predict_type} case for 2021-2023 for {len(list_of_pc6)} pc6's")
   
    # Create dataframe for forecast 2021-2023
    df_forecast = pd.DataFrame()
    df_forecast.loc[:,'ds'] = [2021, 2022, 2023]
    
    #Create output dataframe
    df_output = pd.DataFrame()

    
    # Loop over each pc6 and predict 2021-2023      
    for pc6 in list_of_pc6:
   
        df_predict = pd.DataFrame()
        df_predict['ds'] = df['JAAR'][df['match'] == pc6].copy()
        df_predict['y']  = df['SJV_TOTAAL'][df['match'] == pc6]
        
        # Linear regression
        
        print (f"Forecasting {predict_type} case for 2021-2023 with linear regression model for pc6 {pc6}")

        # Create linear regression object
        regr = linear_model.LinearRegression()

        # Split the data into training/testing sets
        df_predict_X = df_predict['ds'].to_numpy()
        df_predict_Y = df_predict['y'].to_numpy()        
        
        df_predict_X = df_predict_X.reshape(-1, 1)
        df_predict_Y = df_predict_Y.reshape(-1, 1)

        # Train the model using the training sets
        regr.fit(df_predict_X, df_predict_Y)
        
        # Create rows of output dataframe for 2021-2023
        df_copy = pd.DataFrame(df[(df['match'] == pc6) & (df['JAAR'] == 2020)])       
        df_temp = pd.DataFrame(np.repeat(df_copy.values,3,axis=0))
        df_temp.columns = df_copy.columns
        df_temp['JAAR'].iloc[0] = 2021
        df_temp['JAAR'].iloc[1] = 2022
        df_temp['JAAR'].iloc[2] = 2023
        
        # Calculate forecast 
        forecast = regr.predict(df_forecast)
        
        if predict_type == "low":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast[0] * 0.95
           df_temp['SJV_TOTAAL'].iloc[1] = forecast[1] * 0.9
           df_temp['SJV_TOTAAL'].iloc[2] = forecast[2] * 0.85
        elif predict_type == "mid":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast[0]
           df_temp['SJV_TOTAAL'].iloc[1] = forecast[1]
           df_temp['SJV_TOTAAL'].iloc[2] = forecast[2]
        elif predict_type == "high":
           df_temp['SJV_TOTAAL'].iloc[0] = forecast[0] * 1.05
           df_temp['SJV_TOTAAL'].iloc[1] = forecast[1] * 1.1
           df_temp['SJV_TOTAAL'].iloc[2] = forecast[2] * 1.15
        else:
           print(f"Unexpected prediction type {predict_type}")
       
        #print(df_temp)
    
        df_output = df_output.append(df_temp)  
        
    # Delete match column
    del df_output['match']
    
    return df_output

# Set directory

In [262]:
# variables used in script
data_processed_location = '../data/processed'

if 'processed' not in os.getcwd():
    os.chdir(data_processed_location)

# Read dataframes

In [263]:
# kleinverbruikgegevens gegevens inlezen
df_verbruik = pd.read_hdf('kleinverbruikgegevens_data.h5')

#Delete 2021 data by keeping JAAR < 2021
df_verbruik = df_verbruik[df_verbruik['JAAR'] < 2021]

# Fix problem with postcodes
df_verbruik['POSTCODE_VAN'] = df_verbruik['POSTCODE_VAN'].str.replace(" ","")
df_verbruik['POSTCODE_TOT'] = df_verbruik['POSTCODE_TOT'].str.replace(" ","")

# mapping van PC4 buurt naar RES regio
df_pc4_res = pd.read_hdf('pc4_res.h5')


# Merge in RES definitie

In [264]:
# Change type of PC4 for merge
df_verbruik['PC4'] = df_verbruik['PC4'].astype('int64') 

# Merge in RES 
df_verbruik = pd.merge(df_verbruik, df_pc4_res, on=['PC4'], how='left')


# QC plot for 3 PC6's

In [265]:
df1 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '4251AB']
df2 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '6373HA']
df3 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '9998XB']


pc6_1 = alt.Chart(df1.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)
    
pc6_2 = alt.Chart(df2.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_3 = alt.Chart(df3.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)


pc6_1 | pc6_2 | pc6_3

alt.HConcatChart(...)

# Call simple linear regression predictie function voor low, mid, high forecast

In [266]:
df_verbruik_input  =df_verbruik[(df_verbruik['POSTCODE_VAN'] == '4251AB') | (df_verbruik['POSTCODE_VAN'] == '6373HA') | (df_verbruik['POSTCODE_VAN'] == '9998XB')]


# predict low case scenario voor 2021-2023 per PC6 
df_verbruik_low = predict_verbruik(df_verbruik_input, 'low')

# predict mid case scenario voor 2021-2023 per PC6 
df_verbruik_mid = predict_verbruik(df_verbruik_input, 'mid')

# predict high case scenario voor 2021-2023 per PC6 
df_verbruik_high = predict_verbruik(df_verbruik_input, 'high')
                     


Forecasting low case for 2021-2023 for 3 pc6's
Forecasting low case for 2021-2023 with linear regression model for pc6 4251AB-4251AB
Forecasting low case for 2021-2023 with linear regression model for pc6 6373HA-6373HA
Forecasting low case for 2021-2023 with linear regression model for pc6 9998XB-9998XB

Forecasting mid case for 2021-2023 for 3 pc6's
Forecasting mid case for 2021-2023 with linear regression model for pc6 4251AB-4251AB
Forecasting mid case for 2021-2023 with linear regression model for pc6 6373HA-6373HA
Forecasting mid case for 2021-2023 with linear regression model for pc6 9998XB-9998XB

Forecasting high case for 2021-2023 for 3 pc6's
Forecasting high case for 2021-2023 with linear regression model for pc6 4251AB-4251AB
Forecasting high case for 2021-2023 with linear regression model for pc6 6373HA-6373HA
Forecasting high case for 2021-2023 with linear regression model for pc6 9998XB-9998XB


# QC plot for low, mid , high prediction for 3 PC6's¶

In [267]:
df1 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '4251AB']
df2 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '6373HA']
df3 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '9998XB']

df1_low =df_verbruik_low[df_verbruik_low['POSTCODE_VAN'] == '4251AB']
df2_low =df_verbruik_low[df_verbruik_low['POSTCODE_VAN'] == '6373HA']
df3_low =df_verbruik_low[df_verbruik_low['POSTCODE_VAN'] == '9998XB']

df1_mid =df_verbruik_mid[df_verbruik_mid['POSTCODE_VAN'] == '4251AB']
df2_mid =df_verbruik_mid[df_verbruik_mid['POSTCODE_VAN'] == '6373HA']
df3_mid =df_verbruik_mid[df_verbruik_mid['POSTCODE_VAN'] == '9998XB']

df1_high =df_verbruik_high[df_verbruik_high['POSTCODE_VAN'] == '4251AB']
df2_high =df_verbruik_high[df_verbruik_high['POSTCODE_VAN'] == '6373HA']
df3_high =df_verbruik_high[df_verbruik_high['POSTCODE_VAN'] == '9998XB']

pc6_1 = alt.Chart(df1.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)
    
pc6_2 = alt.Chart(df2.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_3 = alt.Chart(df3.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_1_low = alt.Chart(df1_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)
    
pc6_2_low = alt.Chart(df2_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_3_low = alt.Chart(df3_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_1_mid = alt.Chart(df1_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)
    
pc6_2_mid = alt.Chart(df2_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_3_mid = alt.Chart(df3_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_1_high = alt.Chart(df1_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)
    
pc6_2_high = alt.Chart(df2_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)

pc6_3_high = alt.Chart(df3_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)



pc6_1 + pc6_1_low + pc6_1_mid + pc6_1_high| pc6_2 + pc6_2_low + pc6_2_mid + pc6_2_high  | pc6_3  + pc6_3_low + pc6_3_mid + pc6_3_high

alt.HConcatChart(...)

# Call prophet predictie function voor low, mid, high forecast

In [259]:
df_verbruik_input  =df_verbruik[(df_verbruik['POSTCODE_VAN'] == '4251AB') | (df_verbruik['POSTCODE_VAN'] == '6373HA') | (df_verbruik['POSTCODE_VAN'] == '9998XB')]


# predict low case scenario voor 2021-2023 per PC6 
df_verbruik_low = predict_verbruik_prophet(df_verbruik_input, 'low')

# predict mid case scenario voor 2021-2023 per PC6 
df_verbruik_mid = predict_verbruik_prophet(df_verbruik_input, 'mid')

# predict high case scenario voor 2021-2023 per PC6 
df_verbruik_high = predict_verbruik_prophet(df_verbruik_input, 'high')
                     


Forecasting low case for 2021-2023 for 3 pc6's
Forecasting low case for 2021-2023 with prophet model for pc6 4251AB-4251AB
Forecasting low case for 2021-2023 with prophet model for pc6 6373HA-6373HA
Forecasting low case for 2021-2023 with prophet model for pc6 9998XB-9998XB

Forecasting mid case for 2021-2023 for 3 pc6's
Forecasting mid case for 2021-2023 with prophet model for pc6 4251AB-4251AB
Forecasting mid case for 2021-2023 with prophet model for pc6 6373HA-6373HA
Forecasting mid case for 2021-2023 with prophet model for pc6 9998XB-9998XB

Forecasting high case for 2021-2023 for 3 pc6's
Forecasting high case for 2021-2023 with prophet model for pc6 4251AB-4251AB
Forecasting high case for 2021-2023 with prophet model for pc6 6373HA-6373HA
Forecasting high case for 2021-2023 with prophet model for pc6 9998XB-9998XB


In [260]:
df1 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '4251AB']
df2 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '6373HA']
df3 =df_verbruik[df_verbruik['POSTCODE_VAN'] == '9998XB']

df1_low =df_verbruik_low[df_verbruik_low['POSTCODE_VAN'] == '4251AB']
df2_low =df_verbruik_low[df_verbruik_low['POSTCODE_VAN'] == '6373HA']
df3_low =df_verbruik_low[df_verbruik_low['POSTCODE_VAN'] == '9998XB']

df1_mid =df_verbruik_mid[df_verbruik_mid['POSTCODE_VAN'] == '4251AB']
df2_mid =df_verbruik_mid[df_verbruik_mid['POSTCODE_VAN'] == '6373HA']
df3_mid =df_verbruik_mid[df_verbruik_mid['POSTCODE_VAN'] == '9998XB']

df1_high =df_verbruik_high[df_verbruik_high['POSTCODE_VAN'] == '4251AB']
df2_high =df_verbruik_high[df_verbruik_high['POSTCODE_VAN'] == '6373HA']
df3_high =df_verbruik_high[df_verbruik_high['POSTCODE_VAN'] == '9998XB']

pc6_1 = alt.Chart(df1.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)
    
pc6_2 = alt.Chart(df2.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_3 = alt.Chart(df3.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),
                     ).properties(width=250,height=250)

pc6_1_low = alt.Chart(df1_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)
    
pc6_2_low = alt.Chart(df2_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_3_low = alt.Chart(df3_low.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('red')
                     ).properties(width=250,height=250)

pc6_1_mid = alt.Chart(df1_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)
    
pc6_2_mid = alt.Chart(df2_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_3_mid = alt.Chart(df3_mid.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('green')
                     ).properties(width=250,height=250)

pc6_1_high = alt.Chart(df1_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)
    
pc6_2_high = alt.Chart(df2_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)

pc6_3_high = alt.Chart(df3_high.reset_index()).mark_line(clip=True).encode(
                     alt.X('JAAR:T'),
                     alt.Y('SJV_TOTAAL:Q'),color=alt.value('orange')
                     ).properties(width=250,height=250)



pc6_1 + pc6_1_low + pc6_1_mid + pc6_1_high| pc6_2 + pc6_2_low + pc6_2_mid + pc6_2_high  | pc6_3  + pc6_3_low + pc6_3_mid + pc6_3_high

alt.HConcatChart(...)